In [ ]:
# threat_hunting_ai.ipynb (Jupyter Notebook as Python Script)

# Load libraries and API key
import pandas as pd
import numpy as np
import openai
from dotenv import load_dotenv
from sklearn.metrics.pairwise import cosine_similarity
import seaborn as sns
import matplotlib.pyplot as plt
import os

load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY")

# Load log data
log_file = "../logs/sample_logs.csv"
df = pd.read_csv(log_file)
df['message'] = df['message'].astype(str)
texts = df['message'].tolist()

# Embedding function
def get_embedding(text):
    try:
        response = openai.Embedding.create(
            input=text,
            model="text-embedding-3-small"
        )
        return response['data'][0]['embedding']
    except Exception as e:
        print(f"Error embedding text: {text}\n{e}")
        return [0]*1536  # fallback vector

# Generate or load embeddings
embedding_path = "../embeddings/embedded_logs.npy"

if os.path.exists(embedding_path):
    embeddings = np.load(embedding_path)
else:
    embeddings = [get_embedding(msg) for msg in texts]
    np.save(embedding_path, embeddings)

# Define known threat scenario
known_threat = "suspicious admin login from unknown location using ssh"
threat_vector = get_embedding(known_threat)

# Calculate similarity
similarities = cosine_similarity([threat_vector], embeddings)[0]
df['threat_score'] = similarities

# Sort and display top matches
df_sorted = df.sort_values(by='threat_score', ascending=False)
df_sorted[['timestamp', 'username', 'message', 'threat_score']].head(10)

# Plot threat score distribution
sns.histplot(df['threat_score'], bins=50)
plt.title("Distribution of Threat Similarity Scores")
plt.xlabel("Threat Score")
plt.ylabel("Log Count")
plt.show()

# Step 9: Save scored logs
df_sorted.to_csv("../logs/scored_logs.csv", index=False)
print("Scored logs saved to logs/scored_logs.csv")
